https://mitsuba.readthedocs.io/en/latest/src/inverse_rendering/caustics_optimization.html

In [1]:
import os
from os.path import realpath, join

import drjit as dr
import mitsuba as mi

import numpy as np

print(mi.variants())

['scalar_rgb', 'scalar_spectral', 'cuda_ad_rgb', 'llvm_ad_rgb']


In [2]:
#make sure you're not using a scalar varient, they only permit scalar types. this was causing me trouble for a bit
#read this for clarification: https://github.com/mitsuba-renderer/mitsuba3/issues/775
mi.set_variant('llvm_ad_rgb')

setup configs:

In [3]:
SCENE_DIR = realpath('../scenes')

CONFIGS = {
    'wave': {
        'emitter': 'gray',
        'reference': join(SCENE_DIR, 'references/wave-1024.jpg'),
    },
    'qureshi': {
        'emitter': 'gray',
        'reference': join(SCENE_DIR, 'references/f_qureshi-512.jpg'),
    },
    'green': {
        'emitter': 'gray',
        'reference': join(SCENE_DIR, 'references/m_green-512.jpg'),
    },
}

config_name = 'wave'

config = CONFIGS[config_name]
print('selected:', config['reference'])
mi.Bitmap(config['reference'])

selected: C:\Users\Alex\Desktop\exploring-inverse-rendering\scenes\references/wave-1024.jpg


Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [1024, 1024],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma
    uint8 G; // @1, normalized, gamma
    uint8 B; // @2, normalized, gamma
  ],
  data = [ 3 MiB of image data ]
]

setup hparams:

In [4]:
config.update({
    'render_resolution': (128, 128),
    'heightmap_resolution': (512, 512),
    'n_upsampling_steps': 4,
    'spp': 32,
    'max_iterations': 1000,
    'learning_rate': 3e-5,
})

output_dir = realpath(join('.', '4_outputs', config_name))
os.makedirs(output_dir, exist_ok=True)
print('results will be saved to:', output_dir)

results will be saved to: C:\Users\Alex\Desktop\exploring-inverse-rendering\tutorials\4_outputs\wave


function to generate the lens mesh:

In [5]:
def create_flat_lens_mesh(resolution):
    #generate uv coords (texture coordinates)
    U, V = dr.meshgrid(
        dr.linspace(mi.Float, 0, 1, resolution[0]),
        dr.linspace(mi.Float, 0, 1, resolution[1]),
        indexing = 'ij'
    )
    texcoords = mi.Vector2f(U, V)

    #generate vertex coords
    X = 2.0 * (U - 0.5)
    Y = 2.0 * (V - 0.5)
    vertices = mi.Vector3f(X, Y, 0.0)

    #triangulate faces
    faces_x, faces_y, faces_z = [], [], []
    for i in range(resolution[0]-1):
        for j in range(resolution[1]-1):
            v00 = i * resolution[1] + j
            v01 = v00 + 1
            v10 = (i+1) * resolution[1] + j
            v11 = v10 + 1

            faces_x.extend([v00, v01])
            faces_y.extend([v10, v10])
            faces_z.extend([v01, v11])
    
    #assemble face buffer
    faces = mi.Vector3u(faces_x, faces_y, faces_z)

    #make instance of mesh object
    mesh = mi.Mesh("lens-mesh", resolution[0] * resolution[1], len(faces_x), has_vertex_texcoords=True)

    #set mesh buffers
    mesh_params = mi.traverse(mesh)
    mesh_params['vertex_positions'] = dr.ravel(vertices)
    mesh_params['vertex_texcoords'] = dr.ravel(texcoords)
    mesh_params['faces'] = dr.ravel(faces)
    mesh_params.update()

    return mesh

In [6]:
lens_res = config.get('lens_res', config['heightmap_resolution'])
lens_fname = join(output_dir, 'lens_{}_{}.ply'.format(*lens_res))

if not os.path.isfile(lens_fname):
    m = create_flat_lens_mesh(lens_res)
    m.write_ply(lens_fname)
    print('wrote lens mesh ({}x{}) file to: {}'.format(*lens_res, lens_fname))

make emmiter:

In [7]:
emitter = {
    'type': 'directionalarea',
    'radiance': {
        'type': 'spectrum',
        'value': 0.8
    }
}

make integrator:

In [8]:
integrator = {
    'type': 'ptracer',
    'samples_per_pass': 256,
    'max_depth': 4,
    'hide_emitters': False
}

sensor setup:

In [9]:
sensor_to_world = mi.ScalarTransform4f.look_at(
    target=[0, -20, 0],
    origin = [0, -4.65, 0],
    up = [0, 0, 1]
)
resx, resy = config['render_resolution']
sensor = {
    'type': 'perspective',
    'near_clip': 1,
    'far_clip': 1000,
    'fov': 45,
    'to_world': sensor_to_world,

    'sampler': {
        'type': 'independent',
        'sample_count': 512  # Not really used
    },
    'film': {
        'type': 'hdrfilm',
        'width': resx,
        'height': resy,
        'pixel_format': 'rgb',
        'rfilter': {
            # Important: smooth reconstruction filter with a footprint larger than 1 pixel.
            'type': 'gaussian'
        }
    }
}

assemble scene:

In [14]:
scene = {
    'type': 'scene',
    'sensor': sensor,
    'integrator': integrator,
    # Glass BSDF
    'simple-glass': {
        'type': 'dielectric',
        'id': 'simple-glass-bsdf',
        'ext_ior': 'air',
        'int_ior': 1.5,
        'specular_reflectance': { 'type': 'spectrum', 'value': 0 },
    },
    'white-bsdf': {
        'type': 'diffuse',
        'id': 'white-bsdf',
        'reflectance': { 'type': 'rgb', 'value': (1, 1, 1) },
    },
    'black-bsdf': {
        'type': 'diffuse',
        'id': 'black-bsdf',
        'reflectance': { 'type': 'spectrum', 'value': 0 },
    },
    # Receiving plane
    'receiving-plane': {
        'type': 'obj',
        'id': 'receiving-plane',
        'filename': '../scenes/meshes/rectangle.obj',
        'to_world': \
            mi.ScalarTransform4f.look_at(
                target=[0, 1, 0],
                origin=[0, -7, 0],
                up=[0, 0, 1]
            ).scale((5, 5, 5)),
        'bsdf': {'type': 'ref', 'id': 'white-bsdf'},
    },
    # Glass slab, excluding the 'exit' face (added separately below)
    'slab': {
        'type': 'obj',
        'id': 'slab',
        'filename': '../scenes/meshes/slab.obj',
        'to_world': mi.ScalarTransform4f.rotate(axis=(1, 0, 0), angle=90),
        'bsdf': {'type': 'ref', 'id': 'simple-glass'},
    },
    # Glass rectangle, to be optimized
    'lens': {
        'type': 'ply',
        'id': 'lens',
        'filename': lens_fname,
        'to_world': mi.ScalarTransform4f.rotate(axis=(1, 0, 0), angle=90),
        'bsdf': {'type': 'ref', 'id': 'simple-glass'},
    },

    # Directional area emitter placed behind the glass slab
    'focused-emitter-shape': {
        'type': 'obj',
        'filename': '../scenes/meshes/rectangle.obj',
        'to_world': mi.ScalarTransform4f.look_at(
            target=[0, 0, 0],
            origin=[0, 5, 0],
            up=[0, 0, 1]
        ),
        'bsdf': {'type': 'ref', 'id': 'black-bsdf'},
        'focused-emitter': emitter,
    },
}

In [15]:
scene = mi.load_dict(scene)

load reference image

In [16]:
def load_ref_image(config, resolution, output_dir):
    b = mi.Bitmap(config['reference'])
    b = b.convert(mi.Bitmap.PixelFormat.RGB, mi.Bitmap.Float32, False)
    if b.size() != resolution:
        b = b.resample(resolution)

        mi.util.write_bitmap(join(output_dir, 'out_ref.exr'), b)

        print('loaded reference image from:', config['reference'])
        return mi.TensorXf(b)

sensor = scene.sensors()[0]
crop_size = sensor.film().crop_size()
image_ref = load_ref_image(config, crop_size, output_dir=output_dir)

loaded reference image from: C:\Users\Alex\Desktop\exploring-inverse-rendering\scenes\references/wave-1024.jpg
